# 13: Destination in CBD

The models considered to this point include variables that describe the attributes of alternatives, modes, and the characteristics of decision-makers (the work commuters). The mode choice decision also is influenced by variables that describe the context in which the trip is made.

For example, a work trip to the regional central business district (CBD) is more likely to be made by transit than an otherwise similar trip to a suburban work place because the CBD is generally well-served by transit, has more opportunities to make additional stops by walking and is less auto friendly due to congestion and limited and expensive parking. This suggests that the model specification can be enhanced by including variables related to the context of the trip, such as destination zone location.

Model 13 adds the alternative specific CBD dummy variable which indicates whether the destination zone (workplace) is located in the CBD. (pp. 122)

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

for a in [2, 3]:
    m.utility_co[a] = +P("hhinc#2,3") * X("hhinc")
for a in [4, 5, 6]:
    m.utility_co[a] = +P(f"hhinc#{a}") * X("hhinc")

Sometimes we may want to define a part of the utility function that is common across all (or almost all) of the alternatives. We can access a dictionary (more generically called a “mapping”) of alternative codes to alternative names, which can be found via the Dataset.dc.alts_mapping attribute:

In [ ]:
d.dc.alts_mapping

Using this like a standard Python dictionary, we can iterate over all the alternatives, skipping 1, and setting alternative specific constants (ASC’s) for the rest.

In [ ]:
for a, name in d.dc.alts_mapping.items():
    if a == 1:
        continue
    m.utility_co[a] += (
        +P("ASC_" + name)
        + P("vehbywrk_" + name) * X("vehbywrk")
        + P("wkcbd_" + name) * X("wkccbd + wknccbd")
    )

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = (
    +PX("totcost")
    + P("motorized_time") * X("(altid <= 4) * tottime")
    + P("nonmotorized_time") * X("(altid > 4) * tottime")
    + P("motorized_ovtbydist") * X("(altid <= 4) * ovtt/dist")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 13, CBD"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True, method="bhhh")
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*dist.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    ("Ownership", "vehbywrk.*"),
    ("Zonal", "wkcbd.*"),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_Bike": -1.65086990936308,
    "ASC_SR2": -1.6343459309594404,
    "ASC_SR3+": -3.536895423106506,
    "ASC_Transit": -0.20182268467513967,
    "ASC_Walk": 0.08378500087016123,
    "hhinc#2,3": -0.0017353041895238406,
    "hhinc#4": -0.006149027560100709,
    "hhinc#5": -0.011116299683489442,
    "hhinc#6": -0.007835540009972525,
    "motorized_ovtbydist": -0.15011160599438625,
    "motorized_time": -0.028598343252833735,
    "nonmotorized_time": -0.04641360594486308,
    "totcost": -0.0032855739180273234,
    "vehbywrk_Bike": -0.6979736536562279,
    "vehbywrk_SR2": -0.4154516212033234,
    "vehbywrk_SR3+": -0.21205944593613626,
    "vehbywrk_Transit": -0.9109101165995294,
    "vehbywrk_Walk": -0.7194478370164099,
    "wkcbd_Bike": 0.3759603983833265,
    "wkcbd_SR2": 0.2558900818841308,
    "wkcbd_SR3+": 1.0573013291451367,
    "wkcbd_Transit": 1.3562157325427109,
    "wkcbd_Walk": 0.17467164791819956,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )